## Quantum Fourier Transform (QFT)

The Quantum Fourier Transform (QFT) is a core component of many quantum algorithms, 
including Shor’s factoring algorithm, phase estimation, and order-finding. QFT implementation is based on the Coppersmith formula:

$$
|x_{n-1} \dots x_0\rangle \;\mapsto\; 
\bigotimes_{j=0}^{n-1} 
\frac{1}{\sqrt{2}}\left(
|0\rangle + e^{2\pi i \, 0.x_j x_{j-1} \dots x_0}\, |1\rangle
\right),
$$

where $0.x_j x_{j-1} \dots x_0$ denotes the binary fraction. For example,

$$
0.x_2 x_1 x_0 = \frac{x_2}{2} + \frac{x_1}{4} + \frac{x_0}{8}.
$$

<div align="center">
<img src="images/QFT-3-qubits.png">
</div>

Each term in the Coppersmith formula represents a single qubit prepared in a superposition. The relative phase of the $|1\rangle$ component is determined by a binary fraction of the form $0.x_j x_{j-1}\dots x_0$. As we move from the most significant to the least significant qubit, each subsequent qubit carries a phase shift with finer resolution, differing by multiples of $1/2^k$. Thus, the terms are written in order from the largest to the smallest phase contributions. Consequently, on each qubit line, the construction applies an H gate followed by a sequence of 
controlled phase gates ($P$ gates). As we move upward through the circuit, 
each qubit receives one more $P$ gate than the one below it, corresponding to the 
binary digits that control the accumulated phase.

There are two common ways to interpret the Coppersmith formula:  
- Nielsen & Chuang (textbook style): the first Hadamard is placed on the top wire, and controlled-phase gates come from wires below.  
- Bottom-up style: the first Hadamard is placed on the bottom wire, and controlled-phase gates come from wires above.  

In both conventions, it is necessary to add **SWAP gates** at the end of the circuit 
to reverse the qubit order and match the canonical Fourier basis.  

The inverse QFT (IQFT) is obtained by reversing the QFT circuit, inverting each controlled phase, and swapping the qubits back into their original order.  

### Task

- Implement a `qft` function that creates an n-qubit circuit following either a textbook or bottom-up style.  
- Implement the inverse Quantum Fourier Transform circuit `iqft`, based on the previous function.  
- Demonstrate a QFT → IQFT round-trip on at least one basis state and report counts concentrating on the original bitstring.

### Expected Output

- Drawn circuits for the QFT and the inverse QFT.  
- A demonstration of the QFT → IQFT round-trip on a chosen basis state.  
- Measurement results showing counts concentrated on the original bitstring, confirming correctness of the implementation.

### Experimentation

- Test the `qft` function with different numbers of qubits and compare the circuit depth and structure in textbook versus bottom-up style.  
- Apply the QFT to various input states (e.g., computational basis states or superposition states) and observe the resulting measurement distributions.  
- Explore how the presence or absence of the final SWAP gates affects the ordering of the output states.  
- Verify the correctness of the `iqft` by composing QFT → IQFT for different inputs and confirming recovery of the original state.  
- Extend the experiments by using QFT as a subroutine inside other algorithms, such as phase estimation, to observe its role in more complex applications.

---

References: 

[1] D. Coppersmith, *An approximate Fourier transform useful in quantum factoring*, IBM Research Report RC19642, 1994.

[2] M. A. Nielsen and I. L. Chuang, *Quantum Computation and Quantum Information*, 10th Anniversary ed. Cambridge, U.K.: Cambridge Univ. Press, 2010.


In [ ]:
from IPython.display import display

from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.visualization import circuit_drawer
import numpy as np


def qft(n, do_swaps = True, textbook_layout = True, insert_barriers = True):
    """
    Quantum Fourier Transform (QFT).

    Args:
        n: number of qubits
        do_swaps: include final bit-reversal swaps
        textbook_layout:
            True  -> Nielsen & Chuang (top-down): H on top wire first; CPs from lower wires.
            False -> Bottom-up: H on bottom wire first; CPs from wires above (going up).
        insert_barriers: keep textbook layering in the drawing
    """
    qc = QuantumCircuit(n)

    if textbook_layout:
        # Nielsen & Chuang: target goes top -> bottom
        for j in range(n):
            qc.h(j)
            # controls are below the target: j+1..n-1
            for k in range(j + 1, n):
                theta = np.pi / (2 ** (k - j))  # π/2, π/4, ...
                qc.cp(theta, k, j)
            if insert_barriers:
                qc.barrier()
    else:
        # Bottom-up: target goes bottom -> top
        for j in range(n-1, -1, -1): # Loop backwards from top qubit to bottom
            qc.h(j)
            for k in range(j-1, -1, -1): # Loop backwards for controls
                theta = np.pi / (2 ** (j - k))
                qc.cp(theta, k, j) # Control on a higher-index (less significant) qubit
            if insert_barriers:
                qc.barrier()

    if do_swaps:
        for i in range(n // 2):
            qc.swap(i, n - 1 - i)

    return qc

def iqft(n, do_swaps = True, textbook_layout = True, insert_barriers = True):
    """Inverse QFT in the same convention."""
    return qft(n, do_swaps=do_swaps, textbook_layout=textbook_layout, insert_barriers=insert_barriers).inverse()

# -----------------------------------------------
#                main program
# -----------------------------------------------

n = 3
print("Nielsen & Chuang layout:")
qc_nc = qft(n, textbook_layout=True)
display(circuit_drawer(qc_nc, output="mpl"))

print("\nBottom-up layout:")
qc_bu = qft(n, textbook_layout=False)
display(circuit_drawer(qc_bu, output="mpl"))

# -- test the circuits --
n = 4
bitstring = "1101" # basis state to test

qc = QuantumCircuit(n)
for i, bit in enumerate(reversed(bitstring)): # prepare the input state
    if bit == "1":
        qc.x(i)   # flip qubit i to |1>
                
# Use textbook/Nielsen–Chuang layout for both QFT and IQFT
qc.compose(qft(n), inplace=True)
qc.compose(iqft(n), inplace=True)
qc.measure_all()

# --- Simulate and show results ---
simulator = AerSimulator()
result = simulator.run(qc, shots=2048).result()
counts = result.get_counts()

print(f"Round-trip on |{bitstring}⟩ with exact QFT → iQFT: {counts}")
